In [1]:
import sidekit
import os
import sys
import multiprocessing
import matplotlib.pyplot as mpl
import logging
import re
import numpy as np
import itertools

In [2]:
distribNb = 512
rsr2015Path = './'

# Default for RSR2015
audioDir = os.path.join(rsr2015Path , 'data')

# Automatically set the number of parallel process to run.
# The number of threads to run is set equal to the number of cores available
# on the machine minus one or to 1 if the machine has a single core.
nbThread = max(multiprocessing.cpu_count()-1, 1)

In [3]:
import itertools
"""
GMMのファイルパス登録
"""
gmm_wav_path = []
mic = ["WAVES_DT","WAVES_HS"]
per = os.listdir(audioDir+"/WAVES_DT")
for k in ["/".join([audioDir,i,j,"PB"]) for i,j in itertools.product(mic,per)]:
    #print(k[7:])
    if os.listdir(k):
        for l in os.listdir(k):
            gmm_wav_path.append(k[7:]+"/"+l[:-4])
ubmList = gmm_wav_path

In [4]:
"""
ファイルの確認
"""
for i in ubmList:
    if os.path.exists(audioDir+"/"+i+".wav"):
        a = "a"
    else:
        print("ねえ！")

KeyboardInterrupt: 

In [11]:
ubmList[0]

'WAVES_DT/F001/PB/BF001I01_DT'

In [4]:
print('Load task definition')
enroll_idmap = sidekit.IdMap('task/3sesspwd_eval_m_trn.h5')
test_ndx = sidekit.Ndx('task/3sess-pwd_eval_m_ndx.h5')
key = sidekit.Key('task/3sess-pwd_eval_m_key.h5')
#with open('task/ubm_list.txt') as inputFile:
    #ubmList = inputFile.read().split('\n')

Load task definition


In [7]:
print(enroll_idmap)

------------------------------
left ids:array(['1', '1', '1', ..., '100', '100', '100'], dtype='<U255')
right ids:array(['1/1', '1/2', '1/3', ..., '100/98', '100/99', '100/100'],
      dtype='<U255')
seg start:array([None, None, None, ..., None, None, None], dtype=object)
seg stop:array([None, None, None, ..., None, None, None], dtype=object)
------------------------------



In [5]:
logging.info("Initialize FeaturesExtractor")
extractor = sidekit.FeaturesExtractor(audio_filename_structure=audioDir+"/{}.wav",
                                      feature_filename_structure="./features/{}.h5",
                                      sampling_frequency=16000,
                                      lower_frequency=133.3333,
                                      higher_frequency=6955.4976,
                                      filter_bank="log",
                                      filter_bank_size=40,
                                      window_size=0.025,
                                      shift=0.01,
                                      ceps_number=19,
                                      vad="snr",
                                      snr=40,
                                      pre_emphasis=0.97,
                                      save_param=["vad", "energy", "cep"],
                                      keep_all_features=False)


In [12]:
# Get the complete list of features to extract
show_list = np.unique(np.hstack([ubmList]))
channel_list = np.zeros_like(show_list, dtype = int)
logging.info("Extract features and save to disk")
extractor.save_list(show_list=show_list,
                    channel_list=channel_list,
                    num_thread=nbThread)

Process Process-16:
Traceback (most recent call last):
  File "/home/jun/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/jun/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/features_extractor.py", line 750, in save_list
    self.save(show, channel, audio_file, feature_file, noise_file, snr, reverb_file, reverb_level)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/features_extractor.py", line 468, in save
    reverb_level=reverb_level)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/features_extractor.py", line 340, in extract
    self.compressed)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/pytho

  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/frontend/io.py", line 998, in _add_percentile_dataset
    _add_dataset_header(fh, dataset_id, _min_val, _range, _header)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/features_extractor.py", line 468, in save
    reverb_level=reverb_level)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/frontend/io.py", line 972, in _add_dataset_header
    fletcher32=True)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/h5py/_hl/group.py", line 153, in create_dataset
    dsid = dataset.make_new_dset(group, shape, dtype, data, name, **kwds)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/h5py/_hl/group.py", line 153, in create_dataset
    dsid = dataset.make_new_dset(group, shape, dtype, da

ValueError: Unable to create dataset (name already exists)
ValueError: Unable to create dataset (name already exists)
Traceback (most recent call last):
  File "/home/jun/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/jun/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process Process-20:
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/features_extractor.py", line 750, in save_list
    self.save(show, channel, audio_file, feature_file, noise_file, snr, reverb_file, reverb_level)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/features_extractor.py", line 468, in save
    reverb_level=reverb_level)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/features_extractor.py", li

  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/frontend/io.py", line 1236, in _write_show_percentile
    _add_percentile_dataset(fh, show + '/cep', cep)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/frontend/io.py", line 998, in _add_percentile_dataset
    _add_dataset_header(fh, dataset_id, _min_val, _range, _header)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/frontend/io.py", line 972, in _add_dataset_header
    fletcher32=True)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/h5py/_hl/group.py", line 153, in create_dataset
    dsid = dataset.make_new_dset(group, shape, dtype, data, name, **kwds)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/h5py/_hl/dataset.py", line 134, in make_new_dset
    dset_id = h5d.crea

In [6]:
# Create a FeaturesServer to load features and feed the other methods
features_server = sidekit.FeaturesServer(features_extractor=None,
                                         feature_filename_structure="./features/{}.h5",
                                         sources=None,
                                         dataset_list=["energy", "cep", "vad"],
                                         mask=None,
                                         feat_norm="cmvn",
                                         global_cmvn=None,
                                         dct_pca=False,
                                         dct_pca_config=None,
                                         sdc=False,
                                         sdc_config=None,
                                         delta=True,
                                         double_delta=True,
                                         delta_filter=None,
                                         context=None,
                                         traps_dct_nb=None,
                                         rasta=True,
                                         keep_all_features=False)

In [14]:
print('Train the UBM by EM')
# Extract all features and train a GMM without writing to disk
ubm = sidekit.Mixture()
llk = ubm.EM_split(features_server, ubmList, distribNb, num_thread=nbThread, save_partial=True)
ubm.write('gmm/ubm_jnas.h5')

Train the UBM by EM


In [3]:
ubm = sidekit.Mixture("./gmm/ubm_jnas.h5")

In [8]:
print('Compute the sufficient statistics')
# Create a StatServer for the enrollment data and compute the statistics
enroll_stat = sidekit.StatServer(enroll_idmap,
                                 distrib_nb=distribNb,
                                 feature_size=60)
enroll_stat.accumulate_stat(ubm=ubm,
                            feature_server=features_server,
                            seg_indices=range(enroll_stat.segset.shape[0]),
                            num_thread=nbThread)

Compute the sufficient statistics


Process Process-9:
Traceback (most recent call last):
  File "/home/jun/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/jun/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/statserver.py", line 711, in accumulate_stat
    cep, vad = feature_server.load(show, channel=channel)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/features_server.py", line 437, in load
    start=start, stop=stop)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/features_server.py", line 471, in get_features
    h5f = h5py.File(self.feature_filename_structure.format(show), "r")
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/pyt

In [9]:
print('MAP adaptation of the speaker models')
regulation_factor = 16 # MAP regulation factor
enroll_sv = enroll_stat.adapt_mean_map_multisession(ubm, regulation_factor)
enroll_sv.write('data/sv_enroll.h5')

MAP adaptation of the speaker models


/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/statserver.py:1085: RuntimeWarning: invalid value encountered in true_divide
  M = modelStat.stat1 / modelStat.stat0[:, index_map]


In [4]:
enroll_sv = sidekit.StatServer('data/sv_enroll.h5')

In [22]:
print('Compute trial scores')
scores_gmm_ubm = sidekit.gmm_scoring(ubm,
                                     enroll_sv,
                                     test_ndx,
                                     features_server,
                                     num_thread=nbThread)
scores_gmm_ubm.write('scores/scores_gmm-ubm.h5')
scores_gmm_ubm =  sidekit.bosaris.Scores('scores/scores_gmm-ubm.h5')

Compute trial scores


Process Process-20:
Traceback (most recent call last):
  File "/home/jun/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/jun/.pyenv/versions/3.7.4/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/gmm_scoring.py", line 93, in gmm_scoring_singleThread
    cep, _ = feature_server.load(ndx.segset[ts])
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/features_server.py", line 437, in load
    start=start, stop=stop)
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/python3.7/site-packages/sidekit/features_server.py", line 471, in get_features
    h5f = h5py.File(self.feature_filename_structure.format(show), "r")
  File "/home/jun/.local/share/virtualenvs/dnn_voice_change-vie6OHFl/lib/py

In [5]:
sv = enroll_sv.stat1
sv.shape

(100, 30720)

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
pca.fit(sv.T)
ev = pca.components_.T #固有声
ev.shape

(100, 5)

In [7]:
import pandas as pd
import scipy
df = pd.DataFrame(ev)
df_zs = df.apply(scipy.stats.zscore, axis=0)
#df.to_excel('evc.xlsx')
with pd.ExcelWriter('evc_jnas.xlsx') as writer:
    df.to_excel(writer, sheet_name='sheet1')
    df_zs.to_excel(writer, sheet_name='sheet2')

In [8]:
pca.explained_variance_ratio_

array([0.90390365, 0.01637042, 0.00930426, 0.00431846, 0.00409099])

In [9]:
ev_zs = scipy.stats.zscore(ev)

In [14]:
data = []
for x,j in enumerate(ev_zs):
    c = 0
    for i in range(5):
        if j[i] > 0:
            c += 2**(4-i)
    data.append([x+1,c])
list = sorted(data, key=lambda x: x[1])
for i in list:
    print(i)

[40, 3]
[45, 3]
[57, 3]
[3, 4]
[5, 4]
[16, 4]
[17, 4]
[22, 4]
[42, 4]
[46, 4]
[51, 4]
[87, 4]
[89, 4]
[98, 4]
[100, 4]
[4, 5]
[49, 5]
[73, 5]
[14, 6]
[15, 6]
[38, 6]
[43, 6]
[47, 6]
[69, 6]
[70, 6]
[72, 6]
[90, 6]
[1, 7]
[6, 7]
[24, 7]
[41, 7]
[44, 7]
[66, 7]
[86, 7]
[88, 7]
[99, 7]
[35, 10]
[94, 10]
[96, 10]
[2, 11]
[7, 11]
[10, 11]
[11, 11]
[52, 11]
[9, 12]
[21, 12]
[37, 12]
[39, 12]
[59, 12]
[79, 12]
[83, 12]
[12, 13]
[13, 13]
[18, 13]
[19, 13]
[34, 13]
[53, 13]
[54, 13]
[56, 13]
[97, 13]
[25, 14]
[60, 14]
[61, 14]
[62, 14]
[63, 14]
[84, 14]
[33, 15]
[58, 15]
[64, 15]
[65, 15]
[68, 15]
[71, 15]
[93, 15]
[26, 16]
[27, 16]
[28, 16]
[29, 16]
[30, 16]
[31, 16]
[74, 16]
[75, 16]
[76, 16]
[77, 16]
[78, 16]
[80, 16]
[81, 16]
[91, 16]
[32, 23]
[92, 25]
[8, 26]
[82, 26]
[85, 26]
[95, 26]
[55, 27]
[23, 28]
[36, 28]
[50, 29]
[67, 29]
[20, 30]
[48, 31]


In [18]:
female = [2,4,7,8,10,14,15,16,17,18,19,24,25,26,27,29,30,35,36,38,39,40,43,51,53,55,56,57,58,59,60,61,62,63,64,65,66,67,69,72,82,83,84,85,90,91,92,93,94,95,96]
print(len(female))

51


In [19]:
import shutil

for i in range(32):
    if not os.path.exists("./result/a/{}".format(i)):
        os.mkdir("./result/a/{}".format(i))
for i in list:
    if i[0] in female:
        x = "./data/{}/1.wav".format(i[0])
        y = "./result/a/{}/{}.wav".format(i[1],i[0])
        shutil.copyfile(x,y)

In [25]:
prior = sidekit.logit_effective_prior(0.01, 1, 1)
dp = sidekit.DetPlot(window_style='sre10',
                    plot_title="GMM-UBM")
dp.set_system_from_scores(scores_gmm_ubm,
                         key, sys_name="GMM-UBM")
minDCF, Pmiss, Pfa, prebep, EER = sidekit.bosaris.detplot.fast_minDCF(
dp.__tar__[0], dp.__non__[0], prior, normalize=True)
print("UBM-GMM , minDCF = {}, eer = {}".format(minDCF,EER))

UBM-GMM , minDCF = 0.31749524857457234, eer = 0.1455786830998027
